# Return AOS status for a given exposure.

In [ ]:
# Times Square parameters

expId = 2024103100047


In [ ]:
import yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.ts.xml.tables.m1m3 import FATable
from lsst.summit.utils.efdUtils import getEfdData, makeEfdClient

In [ ]:
butler = Butler('/repo/embargo_new', collections=["LSSTComCam/raw/all", "LSSTComCam/calib"])
client = makeEfdClient()

In [ ]:
axials = np.array([[0, 1.601],
               [0.33287, 1.56601],
               [0.65119, 1.46259],
               [0.94104, 1.29524],
               [1.18977, 1.07128],
               [1.38651, 0.8005],
               [1.52264, 0.49474],
               [1.59223, 0.16735],
               [1.59223, -0.16735],
               [1.52264, -0.49474],
               [1.38651, -0.8005],
               [1.18977, -1.07128],
               [0.94104, -1.29524],
               [0.65119, -1.46259],
               [0.33287, -1.56601],
               [0, -1.601],
               [-0.33287, -1.56601],
               [-0.65119, -1.46259],
               [-0.94104, -1.29524],
               [-1.18977, -1.07128],
               [-1.38651, -0.8005],
               [-1.52264, -0.49474],
               [-1.59223, -0.16735],
               [-1.59223, 0.16735],
               [-1.52264, 0.49474],
               [-1.38651, 0.8005],
               [-1.18977, 1.07128],
               [-0.94104, 1.29524],
               [-0.65119, 1.46259],
               [-0.33287, 1.56601],
               [0.1676, 1.27302],
               [0.49137, 1.18626],
               [0.78165, 1.01867],
               [1.01867, 0.78165],
               [1.18626, 0.49137],
               [1.27302, 0.1676],
               [1.27302, -0.1676],
               [1.18626, -0.49137],
               [1.01867, -0.78165],
               [0.78165, -1.01867],
               [0.49137, -1.18626],
               [0.1676, -1.27302],
               [-0.1676, -1.27302],
               [-0.49137, -1.18626],
               [-0.78165, -1.01867],
               [-1.01867, -0.78165],
               [-1.18626, -0.49137],
               [-1.27302, -0.1676 ],
               [-1.27302, 0.1676 ],
               [-1.18626, 0.49137],
               [-1.01867, 0.78165],
               [-0.78165, 1.01867],
               [-0.49137, 1.18626],
               [-0.1676, 1.27302],
               [0, 1.002],
               [0.3427, 0.94157],
               [0.64407, 0.76758],
               [0.86776, 0.501],
               [0.98678, 0.174],
               [0.98678, -0.174],
               [0.86776, -0.501],
               [0.64407, -0.76758],
               [0.3427, -0.94157],
               [0, -1.002],
               [-0.3427, -0.94157],
               [-0.64407, -0.76758],
               [-0.86776, -0.501],
               [-0.98678, -0.174],
               [-0.98678, 0.174],
               [-0.86776, 0.501],
               [-0.64407, 0.76758],
               [-0.3427, 0.94157]])


In [ ]:
# Define the names of the degrees of freedom
DOF_names = []
for component in ["M2", "Cam"]:
    for dof in ["dz", "dx", "dy", "rx", "ry"]:
        DOF_names.append(component+"_"+dof)
for i in range(20):
    DOF_names.append(f"M1M3_B{i+1}")
for i in range(20):
    DOF_names.append(f"M2_B{i+1}")

In [ ]:
def plotM1M3_AOS(df, ax, FATable, zmin=-200, zmax=200):
    ax.set_xlabel("X position (m)")
    ax.set_ylabel("Y position (m)")
    ax.set_xlim(-4.5,4.5)
    ax.set_ylim(-4.5,4.5)
    ax.set_title("M1M3 AOS forces (N)", fontsize=12)
    index = -1
    types = [['SAA','NA', 'o', 'Z'], ['DAA','Y_PLUS', '^', 'Y_PLUS'], ['DAA','Y_MINUS', 'v', 'Y_MINUS'], \
             ['DAA','X_PLUS', '>', 'X_PLUS'], ['DAA','X_MINUS', '<', 'X_MINUS']]

    for [type, orient, marker, label] in types:
        xs = []
        ys = []
        zs = []
        for i in range(len(FATable)):
            x = FATable[i].x_position
            y = FATable[i].y_position
            if FATable[i].actuator_type.name == type and FATable[i].orientation.name == orient:
                xs.append(x)
                ys.append(y)
                name=f"zForces{i}"
                zs.append(df.iloc[index][name])
    im = ax.scatter(xs, ys, marker=marker, c=zs, cmap='RdBu_r', \
                    norm=colors.SymLogNorm(linthresh=zmax/100.0, vmin=zmin, vmax=zmax), \
                     s=50, label=label)
    plt.colorbar(im, ax=ax,fraction=0.055, pad=0.02, cmap='RdBu_r') 
    return

def plotM2_AOS(df, ax, zmin=-200, zmax=200):
    # Get the data from the yaml file
    scale = 2.5
    ax.set_xlim(-scale, scale)
    ax.set_ylim(-scale, scale)
    ax.set_xlabel("X position (m)")
    ax.set_ylabel("Y position (m)")
    ax.set_title("M2 AOS forces (N)", fontsize=12)

    if len(df) == 0:
        ax.text(-1, 0, "Not Available")
        return

    index = -1
    xs = axials[:,0]
    ys = axials[:,1]
    zs = []
    for i in range(len(xs)):
        name=f"axial{i}"
        force = df.iloc[index][name]
        zs.append(force)

    im = ax.scatter(xs, ys, marker='o', c=zs, cmap='RdBu_r', \
                    norm=colors.SymLogNorm(linthresh=zmax/100.0, vmin=zmin, vmax=zmax), \
                     s=80, label="Axial")
    plt.colorbar(im, ax=ax,fraction=0.055, pad=0.0, cmap='RdBu_r')
    return


def getData(client, expRecord, fig, DOF_names):
    ax1 = fig.add_axes([0.10,0.60,0.80,0.30])
    ax2 = fig.add_axes([0.15,0.10,0.20,0.40])
    ax3 = fig.add_axes([0.50,0.10,0.20,0.40])
    ax1.set_axis_off()
    plt.suptitle(f"{expId}", fontsize=24)

    tOpen = Time(expRecord.timespan.begin, scale='tai').utc.isot
    tClose = Time(expRecord.timespan.end, scale='tai').utc.isot
    text1 = f"Topen = {tOpen}, Tclose = {tClose}"
    ax1.text(0.1, 0.9, text1, color='black')
    position = f"Azimuth = {expRecord.azimuth:.2f}, "
    position += f"Elevation = {(90.0 - expRecord.zenith_angle):.2f}, "
    position += f"Rotation = {expRecord.sky_angle:.2f}"
    ax1.text(0.1, 0.7, position, color='black')
    
    hexData = getEfdData(
        client,
        "lsst.sal.MTHexapod.application",
        expRecord=expRecord
    )
    camHex = hexData[hexData['salIndex'] == 1]
    m2Hex = hexData[hexData['salIndex'] == 2]
    names = ['Camera', 'M2']
    yText = [0.5, 0.3]
    for i, hex in enumerate([camHex, m2Hex]):
        textHex = f"{names[i]} hexapod: "
        X = hex.iloc[0]["position0"]
        Y = hex.iloc[0]["position1"]
        Z = hex.iloc[0]["position2"]
        U = hex.iloc[0]["position3"]
        V = hex.iloc[0]["position4"]
        hexPos = f"X={X:.1f}um, Y={Y:.1f}um, Z={Z:.1f}um, U = {U * 3600.0:.1f} arcsec, V = {V * 3600.0:.1f} arcsec"
        textHex += hexPos  
        ax1.text(0.1, yText[i], textHex, color='black')
    
    offsetDOF = getEfdData(
        client,
        "lsst.sal.MTAOS.logevent_degreeOfFreedom",
        expRecord=expRecord,
        prePadding=7200
    )
    
    textBend = "AOS DOF: "
    nModes = 0
    for i in range(50):
        value = offsetDOF.iloc[-1][f"aggregatedDoF{i}"]
        if abs(value) > 1.0E-6:
            textBend += f"{DOF_names[i]} = {value:.2f}, "
            nModes += 1
        if nModes == 5:
            textBend += "\n"
    ax1.text(0.1, 0.1, textBend, color='black')
    
    M1M3_AOS_names = []
    for i in range(156):
        name=f"zForces{i}"
        M1M3_AOS_names.append(name)
    
    M2_AOS_names = []
    for i in range(72):
        name=f"axial{i}"
        M2_AOS_names.append(name)

    M1M3_AOS = getEfdData(
        client,
        "lsst.sal.MTM1M3.command_applyActiveOpticForces",
        expRecord=expRecord,
        columns=M1M3_AOS_names,
        prePadding=7200
    )

    plotM1M3_AOS(M1M3_AOS, ax2, FATable, zmin=-200, zmax=200)

    M2_AOS = getEfdData(
        client,
        "lsst.sal.MTM2.command_applyForces",
        expRecord=expRecord,
        columns=M2_AOS_names,
        prePadding=7200
    )
    
    
    plotM2_AOS(M2_AOS, ax3, zmin=-200, zmax=200)
    
    return fig


In [ ]:
dataId = {'exposure': expId, 'detector': 4, 'instrument': 'LSSTComCam'}
expRecord = butlerUtils.getExpRecordFromDataId(butler, dataId)
fig = plt.figure(figsize=(10,5))
getData(client, expRecord, fig, DOF_names)
plt.show()